# 03. 콘텍스트 밴딧 구현

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        self.bandits = np.array([[0.2, 0, -0.0, -5], [0.1, -5, 1, 0.25], [-5, 5, 5, 5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0, len(self.bandits))
        return self.state
    
    def pullArm(self, action):
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

In [7]:
class agent():
    def __init__(self, lr, s_size, a_size):
        self.state_in = tf.placeholder(shape = [1], dtype = tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in, s_size)
        output = slim.fully_connected(state_in_OH, a_size, biases_initializer = None, \
                                      activation_fn = tf.nn.sigmoid, weights_initializer = tf.ones_initializer())
        self.output = tf.reshape(output, [-1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        self.reward_holder = tf.placeholder(shape = [1], dtype = tf.float32)
        self.action_holder = tf.placeholder(shape = [1], dtype = tf.int32)
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight) * self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
        self.update = optimizer.minimize(self.loss)

In [9]:
tf.reset_default_graph()

cBandit = contextual_bandit()
myAgent = agent(lr = 0.001, s_size = cBandit.num_bandits, a_size = cBandit.num_actions)
weights = tf.trainable_variables()[0]

total_episodes = 10000
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions])
e = 0.1

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit()
        
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action, feed_dict = {myAgent.state_in : [s]})
            
        reward = cBandit.pullArm(action)
        
        feed_dict = {myAgent.reward_holder : [reward], myAgent.action_holder : [action], myAgent.state_in : [s]}
        _, ww = sess.run([myAgent.update, weights], feed_dict = feed_dict)
        
        total_reward[s, action] += reward
        if i% 500 == 0:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward, axis = 1)))
        i += 1
        
for a in range(cBandit.num_bandits):
    print("The agent thinks action " + str(np.argmax(ww[a]) + 1) + " for bandit " + str(a + 1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print("...and it was right!")
    else:
        print("...and it was wrong!")

Mean reward for each of the 3 bandits: [0.   0.   0.25]
Mean reward for each of the 3 bandits: [30.25 40.5  37.5 ]
Mean reward for each of the 3 bandits: [68.   77.75 74.  ]
Mean reward for each of the 3 bandits: [106.25 117.5  108.5 ]
Mean reward for each of the 3 bandits: [148.5  154.   143.75]
Mean reward for each of the 3 bandits: [190.   185.75 181.5 ]
Mean reward for each of the 3 bandits: [228.75 226.75 216.75]
Mean reward for each of the 3 bandits: [268.5  261.5  254.75]
Mean reward for each of the 3 bandits: [310.5  298.25 289.5 ]
Mean reward for each of the 3 bandits: [347.25 338.25 324.75]
Mean reward for each of the 3 bandits: [381.25 375.25 366.25]
Mean reward for each of the 3 bandits: [418.5  411.5  400.75]
Mean reward for each of the 3 bandits: [453.75 450.5  437.5 ]
Mean reward for each of the 3 bandits: [486.75 489.75 473.75]
Mean reward for each of the 3 bandits: [522.75 525.75 508.75]
Mean reward for each of the 3 bandits: [561.5  558.5  542.25]
Mean reward for each